# Trabajar con Procesos (compute)

In [ ]:
# mostrar info de azure-ai-ml
! pip show azure-ai-ml

## Conectar a workspace

In [1]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


Found the config file in: /config.json


Conectado al Workspace: naturgyml4


## Crear un clúster de cómputo

In [2]:
from azure.ai.ml.entities import AmlCompute

# Nombre asignado al clúster de cómputo
cpu_compute_target = "aml-cluster-ric"

try:
    # Veamos si el target de cómputo ya existe
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"Ya existe un clúster llamado {cpu_compute_target}, Lo reutilizaremos."
    )

except Exception:
    print("Creación de un nuevo target de cómputo de CPU...")

    # Creemos el objeto Azure ML Compute con los parámetros previstos
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute es el servicio VM bajo demanda
        type="amlcompute",
        # Familia VM
        size="Standard_D1_v2",
        # Nodos mínimos en ejecución cuando no hay trabajo en ejecución
        min_instances=0,
        # Nodos en el clúster
        max_instances=1,
        # ¿Cuántos segundos se ejecutará el nodo después de la terminación del trabajo?
        idle_time_before_scale_down=120,
        # Dedicated o LowPriority.Este último es más barato, pero hay una posibilidad de terminación del trabajo
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

Creación de un nuevo target de cómputo de CPU...


Después de haber creado un clúster de cómputo, solo puede cambiar la configuración para:

- `min_instances`: Número mínimo de nodos
- `max_instances`: Número máximo de nodos
- `idle_time_before_scale_down`: Tiempo de inactividad antes de la escala hacia abajo

Ahora el clúster de cómputo `aml-cluster` solo puede escalar a un máximo de un nodo.Cambiemos eso a dos, para permitir el cálculo paralelo.

In [4]:
# modificar el número máximo de instancias
cpu_cluster = AmlCompute(
    name=cpu_compute_target,
    max_instances=2
)

ml_client.compute.begin_create_or_update(cpu_cluster)


Cuando se actualiza el clúster de cómputo, se puede verificar su configuración imprimiendo sus atributos.

In [7]:
cpu_cluster = ml_client.compute.get(cpu_compute_target)


print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

AMLCompute with name aml-cluster-ric has a maximum of 2 nodes


## Crear un script para entrenar un modelo

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Ejecutar un job en un clúster de cómputo

In [9]:
from azure.ai.ml import command

# configure job
job = command(
    code="/home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src",
    command="python diabetes-training.py",
    environment="azureml://registries/azureml/environments/sklearn-1.1/versions/4",
    compute=cpu_compute_target,
    display_name="diabetes-train-cluster-ric",
    experiment_name="diabetes-training-ric"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (0.52 MBs): 100%|██████████| 520115/520115 [00:00<00:00, 4015322.00it/s]




Monitor your job at https://ml.azure.com/runs/sad_ocean_y8x6l8sxjc?wsid=/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4&tid=8b85184d-f6f0-4ace-80dd-c3bda5a6c156


In [10]:
ml_client.compute.begin_delete(cpu_compute_target).wait()